# Instrument classifier

## Imports

In [2]:
import scikitplot as skplt
import soundfile as sf
import numpy as np
#import librosa
#import librosa.display
import matplotlib.pyplot as plt
import pandas as pd
import math
import antropy as ant
import seaborn as sns

from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (r2_score,
                             roc_auc_score,
                             classification_report,
                             confusion_matrix,
                             roc_curve,
                             accuracy_score,
                             mean_squared_error)
from sklearn.tree import (DecisionTreeClassifier,
                          plot_tree)
from sklearn.ensemble import RandomForestClassifier

from scipy import signal
from scipy.fft import fft, fftfreq, rfftfreq, rfft
from scipy.stats import norm, kurtosis

from pedalboard import LadderFilter, Pedalboard
from soundfile import SoundFile

## Functions

In [3]:
# Centroid
def centroid(frequency, amplitude):
    centroid = np.sum(amplitude * frequency) / np.sum(amplitude)
    return centroid

# Spread
def spread(frequency, amplitude):
    centroid = np.sum(amplitude * frequency) / np.sum(amplitude)
    spread = math.sqrt(np.sum((frequency - centroid) ** 2 * amplitude) / np.sum(amplitude))
    return spread

# Skewness
def skewness(frequency, amplitude):
    centroid = np.sum(amplitude * frequency) / np.sum(amplitude)
    spread = math.sqrt(np.sum((frequency - centroid) ** 2 * amplitude) / np.sum(amplitude))
    skewness = np.sum((frequency - centroid) ** 3 * amplitude) / (spread ** 3 * np.sum(amplitude))
    return skewness

# Kurtosis
def kurtosis(frequency, amplitude):
    centroid = np.sum(amplitude * frequency) / np.sum(amplitude)
    spread = math.sqrt(np.sum((frequency - centroid) ** 2 * amplitude) / np.sum(amplitude))
    kurtosis = np.sum((frequency - centroid) ** 4 * amplitude) / (spread ** 4 * np.sum(amplitude)) - 3
    return kurtosis

# Entropy
def entropy(amplitude):
    entropy = -np.sum(amplitude * np.log(amplitude)) / np.log(len(amplitude))
    return entropy

# Flatness
def flatness(amplitude):
    flatness = np.exp(np.mean(np.log(amplitude))) / np.mean(amplitude)
    return flatness

# Decrease
def decrease(amplitude):
    first = amplitude[0]
    amplitude = np.delete(amplitude, 0)
    decrease = np.sum((amplitude - first) / (len(amplitude) - 1)) / np.sum(amplitude)
    return decrease

# Slope
def slope(frequency, amplitude):
    slope = sum((frequency - np.mean(frequency)) * (amplitude - np.mean(amplitude))) / np.sum((frequency - np.mean(frequency)) ** 2)
    return slope

# Peak
def peak(frequency, amplitude):
    index = np.argmax(np.array(amplitude))
    peak = frequency[index]
    return peak

# Crest
def crest(frequency, amplitude):
    index = np.argmax(np.array(amplitude))
    peak = frequency[index]
    crest = peak / np.mean(amplitude)
    return crest

# Rolloff
def rolloff(amplitude, treshold):
    if treshold is None:
        treshold = 0.95
    index = np.where(x > amplitude[end] * 0.95, x, np.cumsum(amplitude))
    rolloff = amplitude[index[0]]
    return rolloff

## Instruments

In [9]:
instruments = ["Kick", "Snare", "Hihat", "Guitar", "Bass", "Vox"]
numSounds = 50

## Audio load

In [11]:
generalPath = "../InstrumentsClassifier/Instruments/"

instrumentsAudio = []
for instrument in instruments:
    instrumentPath = generalPath + instrument + "/"
    sound = []
    
    for i in range(numSounds):
        path = instrumentPath + instrument + "_" + str(i+1) + ".wav"
        audio = SoundFile(path)

        if audio.channels == 1:
            samples, sr = sf.read(path)
            sound.append([samples, sr])
        else:
            samples, sr = sf.read(path)
            sound.append([samples[:,0], sr])
            
    instrumentsAudio.append([instrument, sound])

## Pre-processing

### Kicks

In [15]:
kick = []

count = 0
for kick in kicks:
    amplitude = kick[0]
    sr = kick[1]
    time = np.arange(0, kick[0][end], sr)
    
    spectrum = np.abs(rfft(amplitude))
    frequency = rfftfreq(len(amplitude), 1 / sr)
    
    print("Amplitude:", amplitude)
    print("Time:", time)
    print("Spectrum:", spectrum)
    print("Frequency:", frequency)

NameError: name 'kicks' is not defined